## 分布式之前的准备

在深入了解Pytorch分布式计算之前，还需要先对一些基础概念和自己的硬件条件有基础的了解。

### 分布式几个重要概念

- `rank`

多卡分布式训练时，需要多个显卡之间彼此识别，所以需要有一个机制来唯一标示每张显卡，这就是rank

- `world_size`

World size是将参与训练的进程数。每个进程都将被分配一个rank，该rank是一个介于 0 和 world_size - 1 之间的数字，在此作业中是唯一的。它将用作进程标识符，并将用于代替地址，例如，指定应将张量发送到哪个rank（进程）

- `global_rank`

在多节点（或称多机）分布式训练过程中，每台机器和每台机器之间也是不一样的。一张机器上的rank值从0到7分布，这个数值可以称为`local_rank`；而宏观来看，不同机器和不同机器之间的每张卡的标号也是不一样的，因此还需指定`global_rank`。

**由于多节点分布式训练更常见，所以一般来说，`rank`都代指`global_rank`，而将`local_rank`特指为一台机器内8张卡（一般都是8张卡）各自的标号** 

### 查看自己机器对分布式的支持

这部分涉及到的一些方法，在后续的分布式代码中很可能会被反复使用到，可供查阅。

##### 查看单节点上GPU的有关信息

In [1]:
import os
import torch


print(f'PyTorch version: {torch.__version__}')              # Pytorch版本
print(f'GPU available: {torch.cuda.is_available()}')        # CUDA是否可用
print(f'GPU count: {torch.cuda.device_count()}')            # GPU数量
print(f'GPU name: {torch.cuda.get_device_name(0)}')         # GPU名称
print(f'GPU memory: {torch.cuda.get_device_properties(0).total_memory/1024**3:.2f}GB')      # GPU内存
print(f'GCC Version: {os.popen("gcc --version").readlines()[0].strip()}')            # GCC版本
print(f'NVCC Version: {os.popen("nvcc --version").readlines()[3].strip()}')      # NVCC版本

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch version: 1.13.0a0+d0d6b1f
GPU available: True
GPU count: 8
GPU name: Tesla V100-PCIE-32GB
GPU memory: 31.75GB
GCC Version: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
NVCC Version: Cuda compilation tools, release 11.8, V11.8.89


##### 查看机器当前分布式参数。由于我们并未真正初始化一个分布式训练的环境，因而以下函数均只做演示，不能真正地运行。

In [6]:
# 需要先init初始化分布式环境
import torch.distributed as dist

#! 这里只做演示
dist.init_process_group(backend='nccl', init_method='env://')

#! 检查torch.distributed是否可用。
available = torch.distributed.is_available()

#! 获取world size，在不同进程里都是一样的
torch.distributed.get_world_size()

#! 获取rank，每个进程都有自己的序号，各不相同
torch.distributed.get_rank()

#! 获取local_rank。一般情况下，你需要用这个local_rank来手动设置当前模型是跑在当前机器的哪块GPU上面的。
torch.distributed.local_rank()

#! 多进程间阻塞同步：在所有进程或指定进程组中同步，直到所有进程到达这个点。该函数在需要确保多个进程进度相同的时候非常有用，但由于其会导致阻塞降低性能，应只在必要时使用。
torch.distributed.barrier()

#! 输出log、保存模型等操作，需要加上这个判断，否则会出现多个进程同时写入同一个文件的情况
if torch.distributed.get_rank() == 0:
    print('Hello world!')
    torch.save(model.state_dict(), 'model.pth')

##### 其他一些不太常用的查询类API：

In [ ]:
# is_initialized()：检查分布式环境是否已经初始化。
initialized = torch.distributed.is_initialized()  
 
# get_backend()：获取当前使用的分布式后端。
backend = torch.distributed.get_backend()  
 
# is_mpi_available()：检查MPI后端是否可用。
mpi_available = torch.distributed.is_mpi_available()  
 
# is_nccl_available()：检查NCCL后端是否可用。
nccl_available = torch.distributed.is_nccl_available()  
 
# is_gloo_available()：检查Gloo后端是否可用。
gloo_available = torch.distributed.is_gloo_available()  
 
# get_group_name(group)：获取特定进程组的名称。
group_name = torch.distributed.get_group_name(group)  
 
# new_group(ranks=None, backend=None, group_name='')：创建一个新的进程组。
group = torch.distributed.new_group(ranks=[0, 1, 2])